In [1]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_openml

from aif360.datasets import StandardDataset

data = fetch_openml(data_id=1590, as_frame=True)

X_raw = data.data
y_raw = data.target

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

In [2]:
X_unstandardised =  pd.get_dummies(X_raw)
y = (y_raw == ">50K") * 1

In [3]:
# preprocess_data
SCALER = MinMaxScaler()
X = SCALER.fit_transform(X_unstandardised)

In [4]:
X = pd.DataFrame(X, columns=X_unstandardised.columns)

In [6]:
protected_attribute_name = f"sex_Male"

privileged_groups = [{protected_attribute_name: 1}]
unprivileged_groups = [{protected_attribute_name: 0}]


def _create_standard_dataset(
        df: pd.DataFrame,
        protected_attribute_name: str,
        label_name="class",
    ) -> StandardDataset:
        return StandardDataset(
            df,
            label_name=label_name,
            favorable_classes=[1],
            protected_attribute_names=[protected_attribute_name],
            privileged_classes=[[1]],
        )
# Step 1: Create the StandardDataset object
dataset = _create_standard_dataset(pd.concat([X, y], axis=1), protected_attribute_name)

# Step 2: train-test-val :: 15:3:3
dataset_tv, dataset_test = dataset.split([0.7], shuffle=True, seed=42)
dataset_train, dataset_valid = dataset_tv.split([0.7], shuffle=True, seed=42)

In [7]:
LR = LogisticRegression()
LR.fit(dataset_train.features, dataset_train.labels)

c:\Users\I2044\Anaconda3\envs\python38\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\I2044\Anaconda3\envs\python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [8]:
predictions = LR.predict(dataset_valid.features).reshape(-1,1)
probabilities = LR.predict_proba(dataset_valid.features)[:,0].reshape(-1,1)

In [9]:
from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing

In [80]:
EOP = CalibratedEqOddsPostprocessing(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    cost_constraint = "fnr",
    seed=42
)

In [81]:
dataset_valid_pred = dataset_valid.copy()
dataset_valid_pred.labels = predictions
dataset_valid_pred.scores = probabilities

In [82]:
EOP.fit(dataset_valid, dataset_valid_pred)

In [83]:
dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = LR.predict(dataset_test.features).reshape(-1,1)
dataset_test_pred.scores = LR.predict_proba(dataset_test.features)[:,0].reshape(-1,1)

In [84]:
dataset_test_pred_transf = EOP.predict(dataset_test_pred)

In [85]:
from aif360.metrics import ClassificationMetric

metric_unmit = ClassificationMetric(
    dataset_test,
    dataset_test_pred,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

metric_mit = ClassificationMetric(
    dataset_test_pred,
    dataset_test_pred_transf,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [86]:
metric_unmit.disparate_impact()

0.28941538956014623

In [87]:
metric_mit.disparate_impact()

0.9779846067224792